<a href="https://colab.research.google.com/github/janlucasdeinhard/pyfiles/blob/master/10_PROJECT_TIMESERIES/Forecast/CryptoTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


The goal of this notebook is to develop a visualization of useful KPIs for a set of selected crypto currencies. The KPIs should be easy to track and understand, and allow to forecast the near-term earning potential if one had purchased this portfolio. 

In [1]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import warnings,datetime,time,math,itertools,os,sys

import pandas as pd